<a href="https://colab.research.google.com/github/miladqolamee/NeuroMatch_2022_group_project/blob/main/Coding_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is created for exploratory data analysis on Allen 2P dataset by **Pierogi All Stras** 

---
# A brief summary of Allen 2P Experiment Design


- Single and multiple-plane 2 photon calcium imaging was used.
- Exitatory and inhibitory populaiton of nuerons were imaged in different cre-lines.
- Each population was imaged over multiple days and under different experimental conditions (with familiar, novel stimuli and under active and passive veiwing conditions)
- This dataset can be used to evaluate the influence of experience, expectation and task engagemen on neural coding and dynamics.
- Overal, we have 82 mouse, 3021 behavioral sessions and 551 imaging sessions. In total 34619 cells were recorded.
- Mice performed a go/no go visual change detection task. Visual stimui were presented continuously for 1 hour. The start of trial was not indicated and mouse was free to run.

For more information see:
[Link](https://allensdk.readthedocs.io/en/latest/visual_behavior_optical_physiology.html).

# Experimental setup 

<img src="https://github.com/miladqolamee/NeuroMatch_2022_group_project/blob/main/Allen%20viewing%20condition.png?raw=1" alt="Experimental setup" class="bg-primary" width="75%">

---
# A brief summary of Allen 2P Dataset
To access data in the dataset we can instal `allensdk` package. By using `AllenBehaviorOphysProjectCache` object and its methods in this package we can access desired data. 

In the dataset we have 3 tables and 2 datasets streams. We should use informatin inside the table to extract our desired subset of information. These 3 tables are:

1) Behavior Sessions Table: Contains metadata for behavioral sessions (both trianing session and also sessions whcih imaging was perfomed). This table can be accessed by `get_behavior_session_talbe()` method.

2) Ophys Sessions Table: Contains metadata and experimental conditions for all sessions. This talbe can accessed by `get_ophys_session_table()` method.

3) Ophys Experiments Table: Contains metadata and experimental conditions fo all imaging experiment (aka imaging plane). This table can be accessed by `get_experiment_session_table()` method.

For more information see:
[Link](https://github.com/miladqolamee/NeuroMatch_2022_group_project/blob/main/VBP_WhitePaper_SDK_Documentation.pdf).

# Allen 2P dataset terminology

<img src="https://allensdk.readthedocs.io/en/latest/_static/visual_behavior_2p/data_structure.png" alt="dataset terminology" class="bg-primary" width="100%">

---
# First instal and upgrade allensdk package

In [ ]:
# @title Instal allensdk package
# @markdown Run this cell and then restart the Runtime.

# @markdown After running this cell for the first time you don't nee to rerun it.
#pip install allensdk
!pip install allensdk
!pip install hdmf --upgrade

In [ ]:
# @title Upgrade the package
!pip install --upgrade pip
!pip install allensdk


In [2]:
# @title Improt relevant packages
import numpy as np
import pandas as pd
from pathlib import Path
import math

import allensdk
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

## First we need a file path because we need to pass it to cache object to indicate where the file should be downloaded

## By calling `VisualBehaviorOphysProjectCache`, Tables will be downloaded

In [ ]:
# Update this to a valid directory in your filesystem
data_storage_directory = Path("/local1/visual_behavior_ophys_cache_dir")

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

## Behavior Sessions Table

In [4]:
behavior_table = cache.get_behavior_session_table()   # Getting Behavior Session Table
behavior_table.sample(5)                              # Show 5 random observation
print(len(behavior_table))                            # How many rows?
print(len(behavior_table.ophys_session_id.unique()))  # How many unique imaging session

4787
706


## Selecting behavior sessions for VIP and SST nuerons

In [11]:
experiment_variables = behavior_table.columns      # All variables in behavior table

behavior_table.cre_line.unique()    # How many unique cre-line we have

Vip_behavior_talbe = behavior_table[(behavior_table.cre_line.values == 'Vip-IRES-Cre') & 
                                    (behavior_table.ophys_session_id.isnull() == False)]
Sst_behavior_table = behavior_table[(behavior_table.cre_line.values == 'Sst-IRES-Cre') & 
                                    (behavior_table.ophys_session_id.isnull() == False)]


# getting information just for one mouse for simplification and sort its table by date of aquisition
Vip_mouse_id = Vip_behavior_talbe.mouse_id.unique()[0]
Sst_mouse_id = Sst_behavior_table.mouse_id.unique()[0]

this_mouse_Vip_behavior_talbe = behavior_table[(behavior_table.cre_line.values == 'Vip-IRES-Cre') & 
                                    (behavior_table.ophys_session_id.isnull() == False)
                                    & (behavior_table.mouse_id.values == Vip_mouse_id)].sort_values(
                                        by = 'date_of_acquisition')
this_mouse_Sst_behavior_table = behavior_table[(behavior_table.cre_line.values == 'Sst-IRES-Cre') & 
                                    (behavior_table.ophys_session_id.isnull() == False)
                                    & (behavior_table.mouse_id.values == Sst_mouse_id)].sort_values(
                                        by = 'date_of_acquisition')



,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,ophys_experiment_id,ophys_container_id,project_code,date_of_acquisition,session_type,file_id
behavior_session_id,,,,,,,,,,,,,,,,,,,,
792680306,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,148.0,Vip-IRES-Cre,GCaMP6f,1.0,0.0,19.0,1.0,792619807.0,[792815735],[791352433],VisualBehavior,2018-12-10 13:37:29.143,OPHYS_1_images_A,NaN
794968300,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,151.0,Vip-IRES-Cre,GCaMP6f,3.0,0.0,21.0,3.0,794918442.0,[795073741],[791352433],VisualBehavior,2018-12-13 13:33:06.390,OPHYS_3_images_A,NaN
795742990,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,152.0,Vip-IRES-Cre,GCaMP6f,4.0,0.0,0.0,4.0,795625712.0,[795953296],[791352433],VisualBehavior,2018-12-14 12:37:49.365,OPHYS_4_images_B,NaN
796052844,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,153.0,Vip-IRES-Cre,GCaMP6f,5.0,0.0,1.0,5.0,796044280.0,[796108483],[791352433],VisualBehavior,2018-12-15 13:11:14.373,OPHYS_5_images_B_passive,NaN
796251270,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,154.0,Vip-IRES-Cre,GCaMP6f,6.0,0.0,2.0,6.0,796236521.0,[796308505],[791352433],VisualBehavior,2018-12-16 13:15:04.328,OPHYS_6_images_B,NaN
798126878,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,412036,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,156.0,Vip-IRES-Cre,GCaMP6f,2.0,1.0,22.0,7.0,798007990.0,[798404219],[791352433],VisualBehavior,2018-12-18 15:44:40.820,OPHYS_2_images_A_passive,NaN


For this specific Vip mouse we have six imaging session sorted by data of aquisition. In `session_type` column it can be seen what type of image was presented and what kind of task (active or passive ) was performed. 
In the next cell we want to compare the neural response around stimulus omittion during active and passive task.

In [ ]:
#Just a code that might help later on
#M='M'
#behavior_table.query('sex == @ M')
#This way we can specifically ask for the data containing these parameters. the mouse only being male (which is better since female mices' results aren't as reliable)